# Data Cleaning & Alignment

This notebook handles cleaning, standardization, and temporal alignment of
NIFTY Spot and NIFTY Futures intraday data before feature engineering.

## Objectives

The objectives of this notebook are:
- Standardize column names and data types
- Parse and validate datetime fields
- Filter the analysis window to the most recent one-year period
- Align spot and futures data on a common 5-minute timeline
- Save cleaned datasets for downstream feature engineering

## Key Assumptions

- Raw datasets may span multiple years for robustness.
- The analysis window is restricted to the most recent one-year period to align
  with evaluation requirements.
- Spot and futures datasets are assumed to follow the same market trading hours.
- Any missing timestamps will be handled explicitly during alignment.

In [2]:
import pandas as pd
import numpy as np
import os

from IPython.display import display

# Paths
RAW_DATA_PATH = os.path.join("..", "data", "raw")
PROCESSED_DATA_PATH = os.path.join("..", "data", "processed")

os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

# File_paths
spot_path = os.path.join(RAW_DATA_PATH, "nifty_spot_5min.csv")
futures_path = os.path.join(RAW_DATA_PATH, "nifty_futures_5min.csv")

# Load_datasets
spot_df = pd.read_csv(spot_path)
futures_df = pd.read_csv(futures_path)

print("Spot shape:", spot_df.shape)
print("Futures shape:", futures_df.shape)

print("\nSpot columns:")
print(spot_df.columns)

print("\nFutures columns:")
print(futures_df.columns)

display(spot_df.head())


Spot shape: (136202, 6)
Futures shape: (136202, 6)

Spot columns:
Index(['date', 'close', 'high', 'low', 'open', 'volume'], dtype='object')

Futures columns:
Index(['date', 'close', 'high', 'low', 'open', 'volume'], dtype='object')


,date,close,high,low,open,volume
0,2015-01-09 09:15:00+05:30,8301.20,8301.30,8285.45,8285.45,0
1,2015-01-09 09:20:00+05:30,8301.00,8303.00,8293.25,8300.50,0
2,2015-01-09 09:25:00+05:30,8294.15,8302.55,8286.80,8301.65,0
3,2015-01-09 09:30:00+05:30,8288.50,8295.75,8280.65,8294.10,0
4,2015-01-09 09:35:00+05:30,8283.45,8290.45,8278.00,8289.10,0


In [3]:
spot_df.columns = spot_df.columns.str.lower().str.strip()
futures_df.columns = futures_df.columns.str.lower().str.strip()

# Parse datetime column
spot_df["date"] = pd.to_datetime(spot_df["date"])
futures_df["date"] = pd.to_datetime(futures_df["date"])

# Sort by datetime
spot_df = spot_df.sort_values("date").reset_index(drop=True)
futures_df = futures_df.sort_values("date").reset_index(drop=True)

print("Datetime parsing and sorting completed.")
display(spot_df.head())


Datetime parsing and sorting completed.


,date,close,high,low,open,volume
0,2015-01-09 09:15:00+05:30,8301.20,8301.30,8285.45,8285.45,0
1,2015-01-09 09:20:00+05:30,8301.00,8303.00,8293.25,8300.50,0
2,2015-01-09 09:25:00+05:30,8294.15,8302.55,8286.80,8301.65,0
3,2015-01-09 09:30:00+05:30,8288.50,8295.75,8280.65,8294.10,0
4,2015-01-09 09:35:00+05:30,8283.45,8290.45,8278.00,8289.10,0


In [4]:
# Filter to the most recent one-year window
end_date = spot_df["date"].max()
start_date = end_date - pd.DateOffset(years=1)

spot_df_1y = spot_df[spot_df["date"] >= start_date].reset_index(drop=True)
futures_df_1y = futures_df[futures_df["date"] >= start_date].reset_index(drop=True)

print("One-year window:")
print("Start:", start_date)
print("End  :", end_date)

print("\nFiltered Spot shape:", spot_df_1y.shape)
print("Filtered Futures shape:", futures_df_1y.shape)

display(spot_df_1y.head())

One-year window:
Start: 2021-10-21 15:25:00+05:30
End  : 2022-10-21 15:25:00+05:30

Filtered Spot shape: (17488, 6)
Filtered Futures shape: (17488, 6)


,date,close,high,low,open,volume
0,2021-10-21 15:25:00+05:30,18219.60,18222.35,18198.40,18198.40,0
1,2021-10-22 09:15:00+05:30,18249.90,18251.00,18206.15,18230.70,0
2,2021-10-22 09:20:00+05:30,18248.25,18276.00,18225.95,18251.00,0
3,2021-10-22 09:25:00+05:30,18279.15,18283.15,18248.50,18248.50,0
4,2021-10-22 09:30:00+05:30,18295.40,18295.60,18273.85,18280.05,0


In [5]:
# Align spot and futures data on common timestamps
common_timestamps = set(spot_df_1y["date"]).intersection(
    set(futures_df_1y["date"])
)

spot_aligned = spot_df_1y[spot_df_1y["date"].isin(common_timestamps)].reset_index(drop=True)
futures_aligned = futures_df_1y[futures_df_1y["date"].isin(common_timestamps)].reset_index(drop=True)

print("After alignment:")
print("Spot aligned shape:", spot_aligned.shape)
print("Futures aligned shape:", futures_aligned.shape)

# Final sanity check
assert len(spot_aligned) == len(futures_aligned), "Mismatch after alignment"

display(spot_aligned.head())


After alignment:
Spot aligned shape: (17488, 6)
Futures aligned shape: (17488, 6)


,date,close,high,low,open,volume
0,2021-10-21 15:25:00+05:30,18219.60,18222.35,18198.40,18198.40,0
1,2021-10-22 09:15:00+05:30,18249.90,18251.00,18206.15,18230.70,0
2,2021-10-22 09:20:00+05:30,18248.25,18276.00,18225.95,18251.00,0
3,2021-10-22 09:25:00+05:30,18279.15,18283.15,18248.50,18248.50,0
4,2021-10-22 09:30:00+05:30,18295.40,18295.60,18273.85,18280.05,0


In [6]:
#now in this i will save the clean data and alligned datasets
spot_output_path = os.path.join(PROCESSED_DATA_PATH, "spot_cleaned_1y.csv")
futures_output_path = os.path.join(PROCESSED_DATA_PATH, "futures_cleaned_1y.csv")

spot_aligned.to_csv(spot_output_path, index=False)
futures_aligned.to_csv(futures_output_path, index=False)

print("Cleaned datasets saved:")
print(spot_output_path)
print(futures_output_path)

Cleaned datasets saved:
..\data\processed\spot_cleaned_1y.csv
..\data\processed\futures_cleaned_1y.csv
